In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from PIL import Image
from IPython.core.display import HTML
from functools import partial

In [3]:
%reload_ext autoreload
%autoreload 2
from engine.utils_nav import ProgramGenerator, ProgramInterpreter
from prompts.gqa import create_prompt

/home/ziliottf/anaconda3/envs/visprog/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

from engine.step_interpreters_nav import parse_step

eval_ = """EVAL0=EVAL(expr="'STOP' if {LOC0} > 0 else 'NAVIGATE'")"""
stop = """STOP0=STOP(var=EVAL0)"""
# str_2 = """ANSWER2=EVAL(expr="'yes' if {ANSWER0} + {ANSWER1} > 0 else 'no'")"""
dict_ = parse_step(eval_, partial=False)
dict_

In [4]:
question = """ go to [0,10,3]"""
answer= """NAV0=NAVIGATE(image=START, goal='[0,10,3]')
LOC0=LOCATE(pos=NAV0)
EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
STOP0=STOP(var=EVAL0)
"""

image = Image.open('../assets/startnav.png')
image.thumbnail((640,640),Image.Resampling.LANCZOS)
init_state = dict(
    IMAGE=image.convert('RGB')
)

prompter = partial(create_prompt,method='all')
generator = ProgramGenerator(prompter=prompter, debug=True)

interpreter = ProgramInterpreter(dataset='pointnav')
prog,_ = generator.generate(dict(question=question), answer) #fake answer
print(prog)

Registering NAVIGATE step
Registering DETECT step


/home/ziliottf/anaconda3/envs/visprog/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Registering STOP step
Registering LOCATE step
Registering EVAL step
NAV0=NAVIGATE(image=START, goal='[0,10,3]')
LOC0=LOCATE(pos=NAV0)
EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
STOP0=STOP(var=EVAL0)


In [29]:
result, prog_state = interpreter.execute(prog,\
                                        init_state,\
                                        inspect=False)
result

-------------
| Trial n°1 |
-------------
NAVIGATE
LOCATE
EVAL
STOP


True

In [28]:
prog_state

{'IMAGE': <PIL.Image.Image image mode=RGB size=512x512>,
 'NAV0': 1.2,
 'LOC0': False,
 'EVAL0': 'NAVIGATE',
 'STOP0': False}